In [1]:
import transformers
from datasets import Dataset
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

/home/mp/anaconda3/envs/mlops/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/processed/AITA-Reddit-Dataset.csv')

In [3]:
model_name = 'bert-base-uncased'

model = transformers.BertForSequenceClassification.from_pretrained(model_name, num_labels=4, problem_type='multi_label_classification').to('cuda')
tokenizer = transformers.BertTokenizer.from_pretrained(model_name, problem_type='multi_label_classification')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
classifier = transformers.pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

2023-12-07 15:01:40.866243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 15:01:40.866276: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 15:01:40.866307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 15:01:40.872060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 15:01:41.554115: W tensorflow/compiler/

In [5]:
# Create Dataset
base_dataset = Dataset.from_pandas(df)

# Vectorize labels
cols = base_dataset.column_names
base_dataset = base_dataset.map(lambda x: {'labels': [x[c] for c in cols if c != "text"]})

/home/mp/anaconda3/envs/mlops/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [6]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the data (only keep relevant columns)
cols = base_dataset.column_names
cols.remove('labels')
dataset = base_dataset.select(range(1000)).map(tokenize_function, batched=True, remove_columns=cols)

In [7]:
# Create a dataset class with LABEL and TEXT columns
dataset = dataset.train_test_split(test_size=0.2)
validation_test_dataset = dataset['test'].train_test_split(test_size=0.5)
dataset['validation'] = validation_test_dataset['train']
dataset['test'] = validation_test_dataset['test']


In [8]:
# Model evaluation
def compute_metrics(pred):
    print(pred)
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return 0, 0

In [9]:
# finetune the model
training_args = transformers.TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=500,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
)
trainer.train()
    

 13%|█▎        | 102/800 [00:09<01:00, 11.46it/s]

{'loss': 0.5525, 'learning_rate': 1e-05, 'epoch': 0.12}


 25%|██▌       | 202/800 [00:17<00:52, 11.42it/s]

{'loss': 0.3463, 'learning_rate': 2e-05, 'epoch': 0.25}


 38%|███▊      | 302/800 [00:26<00:43, 11.43it/s]

...

 88%|████████▊ | 702/800 [01:05<00:08, 11.22it/s]

{'loss': 0.3399, 'learning_rate': 1.6666666666666667e-05, 'epoch': 0.88}


100%|██████████| 800/800 [01:14<00:00, 10.81it/s]

{'loss': 0.2722, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 74.0231, 'train_samples_per_second': 10.807, 'train_steps_per_second': 10.807, 'train_loss': 0.3611268949508667, 'epoch': 1.0}


TrainOutput(global_step=800, training_loss=0.3611268949508667, metrics={'train_runtime': 74.0231, 'train_samples_per_second': 10.807, 'train_steps_per_second': 10.807, 'train_loss': 0.3611268949508667, 'epoch': 1.0})

In [11]:
predictions = trainer.predict(dataset['test'])

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:02<00:00, 49.37it/s]


In [20]:
predictions.predictions

array([[-6.951932 , -6.9526587,  3.887982 , -3.9039557],
       [-6.993805 , -6.9506974,  3.8821592, -3.8811848],
       [-6.949279 , -6.9518375,  3.884804 , -3.895899 ],
       [-7.0172915, -6.963989 ,  3.8609831, -3.800006 ],
       [-6.9388247, -6.937853 ,  3.881006 , -3.903065 ],
       [-6.951508 , -6.942781 ,  3.869922 , -3.8835871],
       [-6.953735 , -6.9412117,  3.8831904, -3.897611 ],
       [-6.9593654, -6.9604626,  3.8821692, -3.887351 ],
       [-6.983482 , -6.9581547,  3.8893023, -3.8764007],
       [-6.9636717, -6.9517174,  3.8869858, -3.8900533],
       [-6.9386563, -6.941848 ,  3.8772767, -3.8985894],
       [-6.953829 , -6.9445796,  3.8808029, -3.8921132],
       [-6.9491177, -6.945044 ,  3.882238 , -3.8968878],
       [-6.9426227, -6.9393888,  3.8759377, -3.899986 ],
       [-6.9623795, -6.9431124,  3.880493 , -3.8903496],
       [-6.9543047, -6.9564753,  3.8864276, -3.895424 ],
       [-6.9406095, -6.935857 ,  3.8835394, -3.8993976],
       [-7.0017557, -6.962772 ,